In [15]:
import alpaca_trade_api as tradeapi
import requests
from dotenv import load_dotenv
import pandas as pd
import os

In [16]:
load_dotenv(verbose=True)

True

In [17]:
#alpaca_api_key = os.getenv("ALPACA_API_KEY")
#alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca_api_key = "PK2QATFJUTTJTLNCCC9E"
alpaca_secret_key = "OpvkG0g84P35EjDZlJZf0fMdJ4ksJiY6YwxNrN4d"
print(alpaca_api_key)

PK2QATFJUTTJTLNCCC9E


In [18]:
def calculate_sharpe_ratio(df):
    
    df_pct_change = df.pct_change()
    #daily_return
    sharpe_ratio = df_pct_change['close'].mean() / df_pct_change['close'].std()
    
    #annualized sharpe ratio
    return (252 ** 0.5) * sharpe_ratio

In [25]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2011-05-01", tz="America/New_York").isoformat()
today = pd.Timestamp("2020-12-02", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["SPY", "QQQ", "VTI",
           "SPBO", "JNK", "LQD",
           "IYR", "VNQ", "USRT",
           "PSP", "USO", "GLD",
           "MUB", "MUNI", "PZA",
           "VMBS", "MBB", "MBG",
           "TLT", "SCHO", "SHY"
          ]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df


sharpe_ratio_list = []
# for each of the tickers, we will calculate the sharpe ratio and store it in a list of dictionaries 
for ticker in tickers:
    sharpe_ratio = calculate_sharpe_ratio(df_ETFs[ticker])
    sharpe_ratio_list.append({"ticker": ticker, "sharpe_ratio" : sharpe_ratio})

# create a data frame with from the list
df_sharpe_ratio= pd.DataFrame(sharpe_ratio_list)
df_sharpe_ratio


,ticker,sharpe_ratio
0,SPY,0.676037
1,QQQ,0.748989
2,VTI,0.668022
3,SPBO,0.476092
4,JNK,0.306273
5,LQD,0.346619
6,IYR,0.260027
7,VNQ,0.260917
8,USRT,0.165987
9,PSP,0.134122


In [22]:
table = df_sharpe_ratio.pivot(columns='ticker')
table.columns = [col[1] for col in table.columns]
table.head()

,GLD,IYR,JNK,LQD,MBB,MBG,MUB,MUNI,PSP,PZA,...,SCHO,SHY,SPBO,SPY,TLT,USO,USRT,VMBS,VNQ,VTI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.676064,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.66811
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.476383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.306272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
